In [1]:
# ✅ INSTALAR LIBRERÍA
!pip install deap --quiet

import operator
import random
import numpy as np
from deap import base, creator, tools, gp
import matplotlib.pyplot as plt

# ====== TABLA DE VERDAD DEL DECODIFICADOR DE 7 SEGMENTOS ======
# Entradas (A, B, C, D): 4 bits para representar números del 0 al 9
entradas = [list(map(int, format(i, "04b"))) for i in range(10)]

# Salidas esperadas para cada segmento a-g
# Referencia: https://en.wikipedia.org/wiki/Seven-segment_display#Displaying_digits
# Cada fila: [a, b, c, d, e, f, g]
salidas = [
    [1,1,1,1,1,1,0],  # 0
    [0,1,1,0,0,0,0],  # 1
    [1,1,0,1,1,0,1],  # 2
    [1,1,1,1,0,0,1],  # 3
    [0,1,1,0,0,1,1],  # 4
    [1,0,1,1,0,1,1],  # 5
    [1,0,1,1,1,1,1],  # 6
    [1,1,1,0,0,0,0],  # 7
    [1,1,1,1,1,1,1],  # 8
    [1,1,1,1,0,1,1]   # 9
]

# ====== DEFINIR PRIMITIVAS GP ======
pset = gp.PrimitiveSet("MAIN", 4)  # 4 inputs: A, B, C, D

# Operadores lógicos
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_, 1)
pset.addPrimitive(operator.xor, 2)

# Constantes lógicas
pset.addTerminal(0)
pset.addTerminal(1)

# Renombrar inputs
pset.renameArguments(ARG0='A')
pset.renameArguments(ARG1='B')
pset.renameArguments(ARG2='C')
pset.renameArguments(ARG3='D')

# ====== DEFINIR INDIVIDUO Y APTITUD ======
if "FitnessMin" in creator.__dict__:
    del creator.FitnessMin
if "Individual" in creator.__dict__:
    del creator.Individual

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# ====== FUNCIÓN DE APTITUD PARA UN SEGMENTO ======
def evaluar(individuo, segmento_index):
    func = toolbox.compile(expr=individuo)
    error = 0
    for i in range(10):
        entrada = entradas[i]
        salida_esperada = salidas[i][segmento_index]
        try:
            salida = func(*entrada)
            if salida != salida_esperada:
                error += 1
        except:
            error += 1
    return (error,)

toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# ====== EJECUTAR GP PARA CADA SEGMENTO (a-g) ======
MEJORES = []

for segmento in range(7):
    print(f"\n🔧 Evolucionando segmento {chr(97 + segmento)}:")
    toolbox.register("evaluate", evaluar, segmento_index=segmento)

    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    from deap import algorithms
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40,
                                   stats=stats, halloffame=hof, verbose=False)

    best_ind = hof[0]
    print(f"✅ Árbol lógico (error={best_ind.fitness.values[0]}):\n{best_ind}")
    MEJORES.append(best_ind)

# ====== RESULTADO FINAL ======
print("\n📌 Expresiones lógicas evolutivas para los segmentos a-g:")
for i, ind in enumerate(MEJORES):
    print(f"Segmento {chr(97+i)}: {ind}")


🔧 Evolucionando segmento a:
✅ Árbol lógico (error=0.0):
or_(xor(xor(and_(or_(C, D), D), xor(1, C)), and_(B, B)), or_(or_(C, A), not_(not_(0))))

🔧 Evolucionando segmento b:
✅ Árbol lógico (error=0.0):
or_(and_(B, and_(xor(not_(C), D), or_(and_(1, C), not_(C)))), not_(B))

🔧 Evolucionando segmento c:
✅ Árbol lógico (error=0.0):
or_(or_(or_(B, xor(C, or_(not_(0), or_(D, 1)))), 0), or_(B, D))

🔧 Evolucionando segmento d:
✅ Árbol lógico (error=1.0):
not_(and_(and_(D, D), and_(and_(xor(D, xor(C, B)), D), not_(and_(A, A)))))

🔧 Evolucionando segmento e:
✅ Árbol lógico (error=1.0):
and_(not_(D), or_(1, 1))

🔧 Evolucionando segmento f:
✅ Árbol lógico (error=1.0):
and_(xor(not_(B), not_(A)), not_(and_(D, C)))

🔧 Evolucionando segmento g:
✅ Árbol lógico (error=1.0):
or_(not_(or_(B, 1)), or_(or_(C, A), xor(C, B)))

📌 Expresiones lógicas evolutivas para los segmentos a-g:
Segmento a: or_(xor(xor(and_(or_(C, D), D), xor(1, C)), and_(B, B)), or_(or_(C, A), not_(not_(0))))
Segmento b: or_(and_(B, an